In [18]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import math
from collections import defaultdict

In [19]:
data_df_1 = pd.read_parquet('./session100k_img1M.parquet')
edge_df_1 = pd.read_parquet('./session100k_img1M_edges.parquet')

data_df_2 = pd.read_parquet('./session5M_img50M.parquet')
edge_df_2 = pd.read_parquet('./session5M_img50M_edges.parquet')

data_df_3 = pd.read_parquet('./session1M_img10M.parquet')
edge_df_3 = pd.read_parquet('./session1M_img10M_edges.parquet')

In [20]:
def dfs(image_id, visited, cluster, edges):
    stack = [image_id]  # 스택을 생성하고 시작 이미지 ID를 넣습니다.

    while stack:  # 스택이 비어질 때까지 반복합니다.
        current_image_id = stack.pop()  # 스택의 가장 위에 있는 이미지 ID를 꺼냅니다.
        visited.add(current_image_id)  # 방문한 이미지 ID로 표시합니다.
        cluster.append(current_image_id)  # 현재 클러스터에 이미지 ID를 추가합니다.

        if current_image_id in edges:  # 현재 이미지 ID에 연결된 이미지 ID가 있는지 확인합니다.
            for next_image_id in edges[current_image_id]:  # 현재 이미지 ID에 연결된 이미지 ID를 하나씩 확인합니다.
                if next_image_id not in visited:  # 이미 방문하지 않은 이미지 ID인 경우에만 처리합니다.
                    stack.append(next_image_id)  # 다음 이미지 ID를 스택에 추가하여 깊이 우선 탐색을 계속합니다.

In [21]:
def get_session_clusters(data_df, edge_df):
    # 이미지 간 연결 정보를 저장할 딕셔너리
    edges = {}

    # 이미지 간 연결 정보를 저장할 defaultdict를 생성합니다.
    edges = defaultdict(list)

    # data_df를 순회하면서 이미지 간 연결 정보를 추가합니다.
    for _, row in data_df.iterrows():
        session_id = row['session_id']
        img_lst = row['image_id']
        for image_id, next_image_id in zip(img_lst, img_lst[1:]):
            edges[image_id].append(next_image_id)
            edges[next_image_id].append(image_id)

    # edge_df를 순회하면서 이미지 간 연결 정보를 추가합니다.
    for _, row in edge_df.iterrows():
        image_id_from = row['image_id_from']
        image_id_to = row['image_id_to']
        edges[image_id_from].append(image_id_to)
        edges[image_id_to].append(image_id_from)

    # edge_df의 이미지 ID로 구성된 집합을 생성하고, 해당 이미지 ID가 edges에 없는 경우 빈 리스트로 초기화합니다.
    for image_id in set(edge_df['image_id_from']).union(set(edge_df['image_id_to'])):
        edges.setdefault(image_id, [])

    clusters = []  # 클러스터를 저장할 리스트
    visited = set()  # 이미 방문한 이미지 ID를 저장할 집합

    # edges의 모든 이미지 ID를 순회하면서 클러스터를 생성합니다.
    for image_id in edges.keys():
        if image_id not in visited:  # 이미 방문하지 않은 이미지 ID인 경우에만 처리합니다.
            new_cluster = []  # 새로운 클러스터를 생성합니다.
            dfs(image_id, visited, new_cluster, edges)  # DFS를 이용하여 클러스터를 생성합니다.
            clusters.append(new_cluster)  # 생성된 클러스터를 clusters 리스트에 추가합니다.

    return clusters  # 생성된 클러스터 리스트를 반환합니다.


In [22]:
def mapping_img2session(data_df):
        # 이미지 ID를 세션 ID로 매핑하는 딕셔너리 생성
    image_to_session = {}
    for _, row in data_df.iterrows():
        session_id = row['session_id']
        images = row['image_id']
        if isinstance(images, int):
            # 이미지 ID가 하나인 경우
            image_to_session[images] = session_id
        else:
            # 이미지 ID가 리스트인 경우
            for image_id in images:
                image_to_session[image_id] = session_id
    return image_to_session


In [23]:
def get_cluster(data_df, edge_df, get_cluster_idx):
    # 클러스터 생성
    session_cluster = get_session_clusters(data_df, edge_df)

    # 이미지 ID를 세션 ID로 매핑하는 딕셔너리 생성
    image_to_session = mapping_img2session(data_df)

    session_id_lst = []
    image_id_lst = []
    # 클러스터 내의 세션 ID와 이미지 ID 추출하여 결과 데이터 구성
    for cluster in session_cluster:
        session_ids = [image_to_session.get(image_id) for image_id in cluster if image_to_session.get(image_id) is not None]
        image_ids = list(set(cluster))
        session_id_lst.append(session_ids)
        image_id_lst.append(image_ids)

    res = {'session_id': session_id_lst, 'image_id': image_id_lst}
    result_df = pd.DataFrame(res)

    # 결과 데이터프레임 정렬 및 길이 관련 정보 계산
    result_df['session_id_len'] = result_df['session_id'].apply(len)
    result_df = result_df.sort_values(by='session_id_len', ignore_index=True, ascending=False).drop(columns='session_id_len')

    list_len_session = result_df['session_id'].apply(len).tolist()
    len_cluster = len(result_df)
    max_cluster = max(list_len_session)
    mean_cluster = sum(list_len_session) / len_cluster

    get_cluster_idx = 0  # 가져올 클러스터의 인덱스
    idx_df = result_df.loc[get_cluster_idx]
    len_session = len(idx_df['session_id'])
    num_images_in_cluster = len(idx_df['image_id'])
    num_edges_in_cluster = 0

    return len_cluster, max_cluster, mean_cluster, list_len_session, len_session, num_images_in_cluster, num_edges_in_cluster

In [24]:
%%time
get_cluster_idx = 10

len_cluster, max_cluster, mean_cluster, list_len_session, len_session, num_images_in_cluster, num_edges_in_cluster =\
  get_cluster(data_df_1, edge_df_1, get_cluster_idx = get_cluster_idx)

print('session 클러스터의 개수:', len_cluster)
print('최대 클러스터의 sessioin 개수:', max_cluster)
print('클러스터들 평균 session 개수:', max_cluster)
print(get_cluster_idx,'번째 클러스터의 session 개수:', len_session)
print(get_cluster_idx,'번째 클러스터의 image 개수:', num_images_in_cluster)
print(get_cluster_idx,'번째 클러스터의 edge 개수:', num_edges_in_cluster)
print(list_len_session[:30])

session 클러스터의 개수: 1000
최대 클러스터의 sessioin 개수: 9364
클러스터들 평균 session 개수: 9364
10 번째 클러스터의 session 개수: 9364
10 번째 클러스터의 image 개수: 8918
10 번째 클러스터의 edge 개수: 0
[9364, 8689, 6329, 6325, 5845, 5520, 5510, 5442, 5400, 5253, 5119, 5062, 4612, 4609, 4246, 4124, 4028, 3987, 3946, 3866, 3858, 3856, 3838, 3793, 3701, 3672, 3646, 3409, 3349, 3326]
CPU times: user 5.5 s, sys: 162 ms, total: 5.67 s
Wall time: 5.67 s


In [54]:
%%time
get_cluster_idx = 10

len_cluster, max_cluster, mean_cluster, list_len_session, len_session, num_images_in_cluster, num_edges_in_cluster =\
  get_cluster(data_df_2, edge_df_2, get_cluster_idx = get_cluster_idx)

print('session 클러스터의 개수:', len_cluster)
print('최대 클러스터의 sessioin 개수:', max_cluster)
print('클러스터들 평균 session 개수:', max_cluster)
print(get_cluster_idx,'번째 클러스터의 session 개수:', len_session)
print(get_cluster_idx,'번째 클러스터의 image 개수:', num_images_in_cluster)
print(get_cluster_idx,'번째 클러스터의 edge 개수:', num_edges_in_cluster)
print(list_len_session[:30])

session 클러스터의 개수: 50000
최대 클러스터의 sessioin 개수: 1558
클러스터들 평균 session 개수: 1558
10 번째 클러스터의 session 개수: 1558
10 번째 클러스터의 image 개수: 15383
10 번째 클러스터의 edge 개수: 0
[1558, 1068, 1004, 995, 945, 916, 915, 907, 901, 900, 890, 864, 843, 839, 835, 824, 823, 813, 813, 796, 794, 790, 779, 775, 771, 771, 769, 769, 762, 754]
CPU times: user 4min 26s, sys: 33.7 s, total: 4min 59s
Wall time: 5min 15s


In [55]:
%%time
get_cluster_idx = 10

len_cluster, max_cluster, mean_cluster, list_len_session, len_session, num_images_in_cluster, num_edges_in_cluster =\
  get_cluster(data_df_3, edge_df_3, get_cluster_idx = get_cluster_idx)

print('session 클러스터의 개수:', len_cluster)
print('최대 클러스터의 sessioin 개수:', max_cluster)
print('클러스터들 평균 session 개수:', max_cluster)
print(get_cluster_idx,'번째 클러스터의 session 개수:', len_session)
print(get_cluster_idx,'번째 클러스터의 image 개수:', num_images_in_cluster)
print(get_cluster_idx,'번째 클러스터의 edge 개수:', num_edges_in_cluster)
print(list_len_session[:30])

session 클러스터의 개수: 10000
최대 클러스터의 sessioin 개수: 848
클러스터들 평균 session 개수: 848
10 번째 클러스터의 session 개수: 848
10 번째 클러스터의 image 개수: 8555
10 번째 클러스터의 edge 개수: 0
[848, 826, 816, 802, 800, 790, 785, 725, 718, 697, 666, 656, 641, 638, 633, 633, 632, 626, 623, 614, 608, 605, 604, 602, 602, 594, 584, 567, 565, 561]
CPU times: user 50.4 s, sys: 893 ms, total: 51.3 s
Wall time: 51.3 s
